# Mosaico 2008

Script trabajado por Francisco Albornoz (2021-04-13) / Última modificación / Mosaicos Patagonia

2021-08-05 13:07:09 

In [1]:
import ee
import geemap
import geemap.chart as chart
import os
import folium

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [2]:
Map = geemap.Map(center = [-45, -70], zoom = 4)
Map2 = geemap.Map(center = [-45, -70], zoom = 4)
Map3 = geemap.Map(center = [-45, -70], zoom = 4)
Map4 = geemap.Map(center = [-45, -70], zoom = 4)
Map5 = geemap.Map(center = [-45, -70], zoom = 4)
Map6 = geemap.Map(center = [-45, -70], zoom = 4)
Map7 = geemap.Map(center = [-45, -70], zoom = 4)
Map8 = geemap.Map(center = [-45, -70], zoom = 4)
Map9 = geemap.Map(center = [-45, -70], zoom = 4)

## Área de Estudio

In [3]:
work_dir = os.path.expanduser('../../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

## Colors Bars

In [4]:
import geemap.colormaps as cm

In [5]:
# cm.palettes.terrain

In [6]:
color_ndvi = cm.palettes.ndvi

In [7]:
color_ndwi = cm.palettes.ndwi

In [8]:
color_terrain = cm.palettes.terrain

In [9]:
color_glaciar = cm.palettes.PuBuGn_r

In [10]:
color_nubes = cm.palettes.inferno

In [11]:
# cm.list_colormaps()

In [12]:
# cm.plot_colormaps(width=12, height=0.4)

## Funciones

In [13]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [14]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)

In [15]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B2','B4']).rename('NDWI')
    return image.addBands(ndwi)

In [16]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B2','B3']).rename('NDGI')
    return image.addBands(ndgi)

In [17]:
def addSAVI(image):
    savi = image.expression(
        '((b4 - b3)/(b4 + b3 + 0.5))*1.5', {
            'b3':image.select('B3'),
            'b4':image.select('B4')
        }) 
    savi= savi.rename('SAVI')
    return image.addBands(savi)

In [18]:
def addNDSI(image):
    ndsi = image.normalizedDifference(['B2','B5']).rename('NDSI')
    return image.addBands(ndsi)

In [19]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMM'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [20]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [21]:
def Cloud20(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20)) 

In [22]:
def Cloud10(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(10))

In [23]:
def addCloud(image):
    nube = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.addBands(ee.Image(nube.select(['cloud'])).rename('nube'))

## Colecciones

### 2008

In [24]:
start_date = '2007-10-01'
end_date = '2008-03-31'

l5_2008 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [25]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [26]:
vis_nieve = {
  'bands': ['B5', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [27]:
coleccion2008 = l5_2008.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [77]:
QM_NDVI_2008 = coleccion2008.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2008 = coleccion2008.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2008 = coleccion2008.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2008 = coleccion2008.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2008 = coleccion2008.qualityMosaic('NDSI') # Normalized difference snow index

In [78]:
Map.addLayer(QM_NDVI_2008, vizParams, 'QM de NDVI 2008')
Map.addLayer(QM_NDWI_2008, vizParams, 'QM de NDWI 2008')
Map.addLayer(QM_SAVI_2008, vizParams, 'QM de SAVI 2008')
Map.addLayer(QM_NDGI_2008, vis_nieve, 'QM de NDGI 2008')
Map.addLayer(QM_NDSI_2008, vis_nieve, 'QM de NDSI 2008')
Map 

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [30]:
c2008_sn = coleccion2008.map(Cloud20)
NDVI_2008_sn = c2008_sn.qualityMosaic('NDVI')
NDWI_2008_sn = c2008_sn.qualityMosaic('NDWI')
SAVI_2008_sn = c2008_sn.qualityMosaic('SAVI')
NDGI_2008_sn = c2008_sn.qualityMosaic('NDGI')
NDSI_2008_sn = c2008_sn.qualityMosaic('NDSI')

In [31]:
Map2.addLayer(NDVI_2008_sn, vizParams, 'QMsn de NDVI 2008')
Map2.addLayer(NDWI_2008_sn, vizParams, 'QMsn de NDWI 2008')
Map2.addLayer(SAVI_2008_sn, vizParams, 'QMsn de SAVI 2008')
Map2.addLayer(NDGI_2008_sn, vis_nieve, 'QMsn de NDGI 2008')
Map2.addLayer(NDSI_2008_sn, vis_nieve, 'QMsn de NDSI 2008')
# Map2

### 2007

In [32]:
start_date = '2006-10-01'
end_date = '2007-03-31'

l5_2007 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [33]:
coleccion2007 = l5_2007.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [34]:
QM_NDVI_2007 = coleccion2007.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2007 = coleccion2007.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2007 = coleccion2007.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2007 = coleccion2007.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2007 = coleccion2007.qualityMosaic('NDSI') # Normalized difference snow index

In [35]:
Map3.addLayer(QM_NDVI_2007, vizParams, 'QM de NDVI 2007')
Map3.addLayer(QM_NDWI_2007, vizParams, 'QM de NDWI 2007')
Map3.addLayer(QM_SAVI_2007, vizParams, 'QM de SAVI 2007')
Map3.addLayer(QM_NDGI_2007, vis_nieve, 'QM de NDGI 2007')
Map3.addLayer(QM_NDSI_2007, vis_nieve, 'QM de NDSI 2007')
# Map3

In [36]:
c2007_sn = coleccion2007.map(Cloud20)
NDVI_2007_sn = c2007_sn.qualityMosaic('NDVI')
NDWI_2007_sn = c2007_sn.qualityMosaic('NDWI')
SAVI_2007_sn = c2007_sn.qualityMosaic('SAVI')
NDGI_2007_sn = c2007_sn.qualityMosaic('NDGI')
NDSI_2007_sn = c2007_sn.qualityMosaic('NDSI')

In [37]:
Map4.addLayer(NDVI_2007_sn, vizParams, 'QMsn de NDVI 2007')
Map4.addLayer(NDWI_2007_sn, vizParams, 'QMsn de NDWI 2007')
Map4.addLayer(SAVI_2007_sn, vizParams, 'QMsn de SAVI 2007')
Map4.addLayer(NDGI_2007_sn, vis_nieve, 'QMsn de NDGI 2007')
Map4.addLayer(NDSI_2007_sn, vis_nieve, 'QMsn de NDSI 2007')
# Map4

### 2006

In [38]:
start_date = '2005-10-01'
end_date = '2006-03-31'

l5_2006 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [39]:
coleccion2006 = l5_2006.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [40]:
QM_NDVI_2006 = coleccion2006.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2006 = coleccion2006.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2006 = coleccion2006.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2006 = coleccion2006.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2006 = coleccion2006.qualityMosaic('NDSI') # Normalized difference snow index

In [41]:
Map3.addLayer(QM_NDVI_2006, vizParams, 'QM de NDVI 2006')
Map3.addLayer(QM_NDWI_2006, vizParams, 'QM de NDWI 2006')
Map3.addLayer(QM_SAVI_2006, vizParams, 'QM de SAVI 2006')
Map3.addLayer(QM_NDGI_2006, vis_nieve, 'QM de NDGI 2006')
Map3.addLayer(QM_NDSI_2006, vis_nieve, 'QM de NDSI 2006')
# Map3

In [42]:
c2006_sn = coleccion2006.map(Cloud20)
NDVI_2006_sn = c2006_sn.qualityMosaic('NDVI')
NDWI_2006_sn = c2006_sn.qualityMosaic('NDWI')
SAVI_2006_sn = c2006_sn.qualityMosaic('SAVI')
NDGI_2006_sn = c2006_sn.qualityMosaic('NDGI')
NDSI_2006_sn = c2006_sn.qualityMosaic('NDSI')

In [43]:
Map4.addLayer(NDVI_2006_sn, vizParams, 'QMsn de NDVI 2006')
Map4.addLayer(NDWI_2006_sn, vizParams, 'QMsn de NDWI 2006')
Map4.addLayer(SAVI_2006_sn, vizParams, 'QMsn de SAVI 2006')
Map4.addLayer(NDGI_2006_sn, vis_nieve, 'QMsn de NDGI 2006')
Map4.addLayer(NDSI_2006_sn, vis_nieve, 'QMsn de NDSI 2006')
# Map4

### 2008 L7

In [44]:
start_date = '2007-10-01'
end_date = '2008-03-31'

l7_2008 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [45]:
coleccion_l7_2008 = l7_2008.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [46]:
QM_NDVI_2008_l7 = coleccion_l7_2008.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2008_l7 = coleccion_l7_2008.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2008_l7 = coleccion_l7_2008.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2008_l7 = coleccion_l7_2008.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2008_l7 = coleccion_l7_2008.qualityMosaic('NDSI') # Normalized difference snow index

In [47]:
Map.addLayer(QM_NDVI_2008_l7, vizParams, 'QM de NDVI 2008_l7')
Map.addLayer(QM_NDWI_2008_l7, vizParams, 'QM de NDWI 2008_l7')
Map.addLayer(QM_SAVI_2008_l7, vizParams, 'QM de SAVI 2008_l7')
Map.addLayer(QM_NDGI_2008_l7, vis_nieve, 'QM de NDGI 2008_l7')
Map.addLayer(QM_NDSI_2008_l7, vis_nieve, 'QM de NDSI 2008_l7')
# Map

In [48]:
c2008_l7_sn = coleccion_l7_2008.map(Cloud20)
NDVI_2008_l7_sn = c2008_l7_sn.qualityMosaic('NDVI')
NDWI_2008_l7_sn = c2008_l7_sn.qualityMosaic('NDWI')
SAVI_2008_l7_sn = c2008_l7_sn.qualityMosaic('SAVI')
NDGI_2008_l7_sn = c2008_l7_sn.qualityMosaic('NDGI')
NDSI_2008_l7_sn = c2008_l7_sn.qualityMosaic('NDSI')

In [49]:
Map2.addLayer(NDVI_2008_l7_sn, vizParams, 'QMsn de NDVI 2008_l7')
Map2.addLayer(NDWI_2008_l7_sn, vizParams, 'QMsn de NDWI 2008_l7')
Map2.addLayer(SAVI_2008_l7_sn, vizParams, 'QMsn de SAVI 2008_l7')
Map2.addLayer(NDGI_2008_l7_sn, vis_nieve, 'QMsn de NDGI 2008_l7')
Map2.addLayer(NDSI_2008_l7_sn, vis_nieve, 'QMsn de NDSI 2008_l7')
# Map2

### 2007 L7

In [50]:
start_date = '2006-10-01'
end_date = '2007-03-31'

l7_2007 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [51]:
coleccion_l7_2007 = l7_2007.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [52]:
QM_NDVI_2007_l7 = coleccion_l7_2007.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2007_l7 = coleccion_l7_2007.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2007_l7 = coleccion_l7_2007.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2007_l7 = coleccion_l7_2007.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2007_l7 = coleccion_l7_2007.qualityMosaic('NDSI') # Normalized difference snow index

In [53]:
Map5.addLayer(QM_NDVI_2007_l7, vizParams, 'QM de NDVI 2007_l7')
Map5.addLayer(QM_NDWI_2007_l7, vizParams, 'QM de NDWI 2007_l7')
Map5.addLayer(QM_SAVI_2007_l7, vizParams, 'QM de SAVI 2007_l7')
Map5.addLayer(QM_NDGI_2007_l7, vis_nieve, 'QM de NDGI 2007_l7')
Map5.addLayer(QM_NDSI_2007_l7, vis_nieve, 'QM de NDSI 2007_l7')
# Map5

In [54]:
c2007_l7_sn = coleccion_l7_2007.map(Cloud20)
NDVI_2007_l7_sn = c2007_l7_sn.qualityMosaic('NDVI')
NDWI_2007_l7_sn = c2007_l7_sn.qualityMosaic('NDWI')
SAVI_2007_l7_sn = c2007_l7_sn.qualityMosaic('SAVI')
NDGI_2007_l7_sn = c2007_l7_sn.qualityMosaic('NDGI')
NDSI_2007_l7_sn = c2007_l7_sn.qualityMosaic('NDSI')

In [55]:
Map6.addLayer(NDVI_2007_l7_sn, vizParams, 'QMsn de NDVI 2007_l7')
Map6.addLayer(NDWI_2007_l7_sn, vizParams, 'QMsn de NDWI 2007_l7')
Map6.addLayer(SAVI_2007_l7_sn, vizParams, 'QMsn de SAVI 2007_l7')
Map6.addLayer(NDGI_2007_l7_sn, vis_nieve, 'QMsn de NDGI 2007_l7')
Map6.addLayer(NDSI_2007_l7_sn, vis_nieve, 'QMsn de NDSI 2007_l7')
# Map6

## Indices por colección

### Índices 2008

In [56]:
NDVI_2008 = NDVI_2008_sn.select('NDVI')
NDWI_2008 = NDWI_2008_sn.select('NDWI') 
SAVI_2008 = SAVI_2008_sn.select('SAVI')
NDGI_2008 = NDGI_2008_sn.select('NDGI')
NDSI_2008 = NDSI_2008_sn.select('NDSI')

In [57]:
Map7.addLayer(NDVI_2008, {'palette': color_terrain}, 'QMsn de NDVI 2008')
Map7.addLayer(NDWI_2008, {'palette': color_ndwi}, 'QMsn de NDWI 2008')
Map7.addLayer(SAVI_2008, {'palette': color_ndvi}, 'QMsn de SAVI 2008')
# Map7.addLayer(NDGI_2008, {'palette': color_glaciar}, 'QMsn de NDGI 2008')
# Map7.addLayer(NDSI_2008, {'palette': color_ndwi}, 'QMsn de NDSI 2008')
Map7

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Índices 2007

In [58]:
NDVI_2007 = NDVI_2007_sn.select('NDVI')
NDWI_2007 = NDWI_2007_sn.select('NDWI') 
SAVI_2007 = SAVI_2007_sn.select('SAVI')
NDGI_2007 = NDGI_2007_sn.select('NDGI')
NDSI_2007 = NDSI_2007_sn.select('NDSI')

In [59]:
Map7.addLayer(NDVI_2007, {'palette': color_terrain}, 'QMsn de NDVI 2007')
Map7.addLayer(NDWI_2007, {'palette': color_ndwi}, 'QMsn de NDWI 2007')
Map7.addLayer(SAVI_2007, {'palette': color_ndvi}, 'QMsn de SAVI 2007')
# Map7.addLayer(NDGI_2007, {'palette': color_glaciar}, 'QMsn de NDGI 2007')
# Map7.addLayer(NDSI_2007, {'palette': color_ndwi}, 'QMsn de NDSI 2007')
Map7

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Índices 2006

In [60]:
NDVI_2006 = NDVI_2006_sn.select('NDVI')
NDWI_2006 = NDWI_2006_sn.select('NDWI') 
SAVI_2006 = SAVI_2006_sn.select('SAVI')
NDGI_2006 = NDGI_2006_sn.select('NDGI')
NDSI_2006 = NDSI_2006_sn.select('NDSI')

In [61]:
# Map8.addLayer(NDVI_2006, {'palette': color_terrain}, 'QMsn de NDVI 2006')
# Map8.addLayer(NDWI_2006, {'palette': color_ndwi}, 'QMsn de NDWI 2006')
# Map8.addLayer(SAVI_2006, {'palette': color_ndvi}, 'QMsn de SAVI 2006')
# Map8.addLayer(NDGI_2006, {'palette': color_glaciar}, 'QMsn de NDGI 2006')
# Map8.addLayer(NDSI_2006, {'palette': color_ndwi}, 'QMsn de NDSI 2006')
# Map8

### Índices 2008 L7

In [62]:
NDVI_2008_l7 = NDVI_2008_l7_sn.select('NDVI')
NDWI_2008_l7 = NDWI_2008_l7_sn.select('NDWI')
SAVI_2008_l7 = SAVI_2008_l7_sn.select('SAVI')
NDGI_2008_l7 = NDGI_2008_l7_sn.select('NDGI')
NDSI_2008_l7 = NDSI_2008_l7_sn.select('NDSI')

In [63]:
# Map8.addLayer(NDVI_2008_l7, {'palette': color_terrain}, 'QMsn de NDVI 2008_l7')
# Map8.addLayer(NDWI_2008_l7, {'palette': color_ndwi}, 'QMsn de NDWI 2008_l7')
# Map8.addLayer(SAVI_2008_l7, {'palette': color_ndvi}, 'QMsn de SAVI 2008_l7')
# Map8.addLayer(NDGI_2008_l7, {'palette': color_glaciar}, 'QMsn de NDGI 2008_l7')
# Map8.addLayer(NDSI_2008_l7, {'palette': color_ndwi}, 'QMsn de NDSI 2008_l7')
# Map8

## Generación de Mosaico

In [64]:
NDVI_2008_landsat = QM_NDVI_2008.set('SENSOR_ID','TM')

In [88]:
properties = QM_NDVI_2007.get('CLOUD_COVER')


In [92]:
print(QM_NDVI_2008)

ee.Image({
  "functionInvocationValue": {
    "functionName": "ImageCollection.qualityMosaic",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.map",
          "arguments": {
            "baseAlgorithm": {
              "functionDefinitionValue": {
                "argumentNames": [
                  "_MAPPING_VAR_0_0"
                ],
                "body": {
                  "functionInvocationValue": {
                    "functionName": "Image.addBands",
                    "arguments": {
                      "dstImg": {
                        "argumentReference": "_MAPPING_VAR_0_0"
                      },
                      "srcImg": {
                        "functionInvocationValue": {
                          "functionName": "Image.rename",
                          "arguments": {
                            "input": {
                              "functionInvocationValue": {
                        

In [71]:
asdf = QM_NDVI_2008.propertyNames()

In [65]:
cloud_2008 = ee.Algorithms.Landsat.simpleCloudScore(NDVI_2008_landsat)

In [66]:
cloud_2008 = cloud_2008.select('cloud')

In [67]:
palette_nubes = ['#1a9850', '#66bd63', '#a6d96a', '#d9ef8b', '#fee08b', '#fdae61', '#f46d43', '#d73027']

In [68]:
resultados= [cloud_2008, QM_NDVI_2008]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes 2008',
    'QM NDVI 2008']

In [69]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

EEException: Landsat.simpleCloudScore: Image is not a Landsat scene or is missing SPACECRAFT_ID metadata.

In [ ]:
def llenado(image):
    return image.where(cloud_2008.gt(20),NDVI_2008_sn)

In [ ]:
img1 = llenado(QM_NDVI_2021)

In [ ]:
img1_landsat = img1.set('SENSOR_ID', 'OLI_TIRS')
cloud_img1 = ee.Algorithms.Landsat.simpleCloudScore(img1_landsat)
cloud_img1 = cloud_img1.select('cloud')

In [ ]:
resultados= [cloud_img1, img1]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 1',
    'Mosaico 1']

In [ ]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [ ]:
def llenado2(image):
    return image.where(cloud_img1.gt(20), NDVI_2020_sn)

In [ ]:
img2 = llenado2(img1)

In [ ]:
img2_landsat = img2.set('SENSOR_ID', 'OLI_TIRS')
cloud_img2 = ee.Algorithms.Landsat.simpleCloudScore(img2_landsat)
cloud_img2 = cloud_img2.select('cloud')

In [ ]:
resultados= [cloud_img2, img2]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 2',
    'Mosaico 2']

In [ ]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [ ]:
def llenado3(image):
    return image.where(cloud_img2.gt(20), NDVI_2019_sn)

In [ ]:
img3 = llenado3(img2)

In [ ]:
img3_landsat = img3.set('SENSOR_ID', 'OLI_TIRS')
cloud_img3 = ee.Algorithms.Landsat.simpleCloudScore(img3_landsat)
cloud_img3 = cloud_img3.select('cloud')

In [ ]:
resultados= [cloud_img3, img3]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 3',
    'Mosaico 3']

In [ ]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [ ]:
def llenado4(image):
    return image.where(cloud_img3.gt(20), NDVI_2018_sn)

In [ ]:
img4 = llenado4(img3)

In [ ]:
img4_landsat = img4.set('SENSOR_ID', 'OLI_TIRS')
cloud_img4 = ee.Algorithms.Landsat.simpleCloudScore(img4_landsat)
cloud_img4 = cloud_img4.select('cloud')

In [ ]:
resultados= [cloud_img4, img4]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 4',
    'Mosaico 4']

In [ ]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

In [ ]:
def llenado5(image):
    return image.where(cloud_img4.gt(20), NDVI_2017_sn)

In [ ]:
img5 = llenado5(img4)

In [ ]:
img5_landsat = img5.set('SENSOR_ID', 'OLI_TIRS')
cloud_img5 = ee.Algorithms.Landsat.simpleCloudScore(img5_landsat)
cloud_img5 = cloud_img5.select('cloud')

In [ ]:
resultados= [cloud_img5, img5]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 5',
    'Mosaico 5']

In [ ]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")